In [1]:
library(tidyverse)
library(magrittr)
library(patchwork)

theme_set(theme_bw())

library(SingleCellExperiment)
library(batchelor)
library(multisce)
library(scutility)

seed <- 124
set.seed(seed)

Warning message:
“package ‘tibble’ was built under R version 4.3.2”
Warning message:
“package ‘readr’ was built under R version 4.3.2”
Warning message:
“package ‘forcats’ was built under R version 4.3.2”
Warning message:
“package ‘lubridate’ was built under R version 4.3.2”
── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts 

In [2]:
path_multisce <- dir(here::here("data", "multisce"), pattern="_", full.names=TRUE) %>% setNames(., basename(.))
path_multisce %<>% .[grep("^skin_", names(.), invert=TRUE)]
path_multisce

Gaydosik2022_HC1_HC2 
       "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_HC1_HC2" 
                                                             Gaydosik2022_HC3_HC4 
       "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_HC3_HC4" 
                                                                Gaydosik2022_MF17 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_MF17" 
                                                                Gaydosik2022_MF18 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_MF18" 
                                                                Gaydosik2022_MF19 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_MF19" 
                                                                Gaydosik2022_MF21 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_MF21" 
                                                                Gaydosik2022_MF24 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2022_MF24" 
                                                                 Gaydosik2023_HC1 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC1" 
                                                                 Gaydosik2023_HC2 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC2" 
                                                                 Gaydosik2023_HC3 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC3" 
                                                                 Gaydosik2023_HC4 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC4" 
                                                                 Gaydosik2023_HC5 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC5" 
                                                                 Gaydosik2023_HC6 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC6" 
                                                                 Gaydosik2023_HC7 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC7" 
                                                                 Gaydosik2023_HC8 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC8" 
                                                                 Gaydosik2023_HC9 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_HC9" 
                                                                Gaydosik2023_MF12 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_MF12" 
                                                                 Gaydosik2023_MF6 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_MF6" 
                                                                 Gaydosik2023_MF8 
           "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Gaydosik2023_MF8" 
                                                                Rindler2021a_skin 
          "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Rindler2021a_skin" 
                                                      Rindler2021b_MF309_followup 
"/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Rindler2021b_MF309_followup" 
                                                         Rindler2021b_MF309_thick 
   "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Rindler2021b_MF309_thick" 
                                                          Rindler2021b_MF309_thin 
    "/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Rindler2021b_MF309_thin" 
                                                      Rindler2021b_MF311_followup 
"/home/tbb/git/CTCL_KC_SA_presentation/data/multisce/Rindler2021b_MF311_followup" 
                                                         Rindler2021b_MF311_t

In [3]:
runs <- names(path_multisce) %>% factor()

# Merge cell annotations

In [4]:
coldata_list <- pmap(list(path_multisce, as.numeric(runs), runs), function(path, runID, run){
    coldata <- readRDS(here::here(path, "coldata.rds"))
    study <- gsub("^([A-Za-z0-9]+)_.*", "\\1", run)
    
    donor <- gsub(".*_(MF|P|PT|CTCL)([0-9]+).*", "\\1\\2", run)
    if(donor == run){
        donor <- gsub(".*_(SZ[0-9]+).*", "\\1", run)
    }
    if(donor != run){
        donor %<>% paste(study, ., sep="_")
    }
    
    rownames(coldata) %<>% paste(runID, sep="_")
    coldata$study <- study
    coldata$donor <- donor    
    
    return(coldata)
})

In [5]:
# Get a collection of included colnames but remove "run" as we will add this below
coldata_names <- map(coldata_list, colnames) %>% Reduce(union, .) %>% setdiff(c("run"))

In [6]:
coldata <- coldata_list %>% map(~ .x %>% as.data.frame() %>% .[,intersect(colnames(.), coldata_names)]) %>% bind_rows(.id="run")
coldata$run %<>% factor()
head(coldata)

,run,sum,detected,subsets_Mito_sum,subsets_Mito_detected,subsets_Mito_percent,total,sizeFactor,louvain,cluster,⋯,Multi.2.j_gene,Multi.2.c_gene,Multi.2.cdr3,Multi.2.cdr3_nt,Multi.2.umis,Multi.2.reads,Multi.2.high_confidence,Multi.2.productive,Multi.2.full_length,Multi.2.raw_clonotype_id
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<lgl>,<lgl>,<lgl>,<fct>
AAACCTGAGACAAGCC-1_1,Gaydosik2022_HC1_HC2,9979,3090,225,12,2.254735,9979,2.3317655,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACCTGAGCTAGTTC-1_1,Gaydosik2022_HC1_HC2,820,474,48,11,5.853659,820,0.1916071,2,2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACCTGCAACGATGG-1_1,Gaydosik2022_HC1_HC2,4914,1745,320,12,6.512007,4914,1.1482409,3,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACCTGCAATGAATG-1_1,Gaydosik2022_HC1_HC2,1022,390,43,11,4.207436,1022,0.2388079,4,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACCTGTCTGGTTCC-1_1,Gaydosik2022_HC1_HC2,15873,2602,600,13,3.780004,15873,3.7090003,4,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AAACGGGAGACGCAAC-1_1,Gaydosik2022_HC1_HC2,37802,4347,2522,12,6.671605,37802,8.8330895,4,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Merge RNA counts

In [7]:
counts_list <- pmap(list(path_multisce, as.numeric(runs), runs), function(path, runID, run){
    data <- counts(readRDS(here::here(path, "sce", "RNA.rds")))
    colnames(data) %<>% paste(runID, sep="_")
    
    return(data)
})

In [8]:
# The Gaydosik 2019 study is using an old reference annotation - to allow integration, we only use genes also included in GRCh38
rownames_include <- rownames(counts_list[["Gaydosik2022_MF17"]])

data_counts <- counts_list %>% map(scutility::subset_matrix, features=rownames_include) %>% Reduce(cbind, .)

Some annotation needs to be adjusted to be consistent

In [9]:
coldata$tissue[is.na(coldata$tissue)] <- "Skin"

In [10]:
#Rindler2021a only has data from a single donor
coldata$donor[coldata$study == "Rindler2021a"] <- "Rindler2021a_MF1"

In [11]:
coldata$sample[is.na(coldata$sample)] <- coldata$run[is.na(coldata$sample)]

# Identify malignant clone
Using scTCR-seq information

In [12]:
coldata$TRB.1.cdr3[coldata$TRB.1.cdr3 == "None"] <- NA
coldata$TRA.1.cdr3[coldata$TRA.1.cdr3 == "None"] <- NA

In [32]:
TRB <- coldata %>% 
    group_by(donor) %>% 
    mutate(TRB_top1=fct_lump_n(TRB.1.cdr3, n=1, ties.method="first", other_level="Other")) %>% 
    filter(!is.na(TRB_top1) & !TRB_top1 %in% c("Other")) %>% 
    mutate(TRB_top1_count=n()) %>% 
    #filter(!is.na(TRA.1.cdr3) & !TRA.1.cdr3 %in% c("None","Other")) %>% 
    group_by(donor, TRB_top1, TRB_top1_count, TRA.1.cdr3) %>% summarize(TRA_count=n()) %>% 
    filter(TRA_count > 2) %>% 
    mutate(TRA_freq=TRA_count/TRB_top1_count) %>% slice_max(n=10, order_by=TRA_freq)

TRB[TRB$donor == "Gaydosik2022_MF17", ]

`summarise()` has grouped output by 'donor', 'TRB_top1', 'TRB_top1_count'. You can override using the `.groups` argument.


donor,TRB_top1,TRB_top1_count,TRA.1.cdr3,TRA_count,TRA_freq
<chr>,<fct>,<int>,<chr>,<int>,<dbl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333


In [33]:
TRA <- coldata %>% 
    group_by(donor) %>% 
    mutate(TRA_top1=fct_lump_n(TRA.1.cdr3, n=1, ties.method="first", other_level="Other")) %>% 
    filter(!is.na(TRA_top1) & !TRA_top1 %in% c("Other")) %>% 
    mutate(TRA_top1_count=n()) %>% 
    #filter(!is.na(TRB.1.cdr3) & !TRB.1.cdr3 %in% c("None","Other")) %>% 
    group_by(donor, TRA_top1, TRA_top1_count, TRB.1.cdr3) %>% summarize(TRB_count=n()) %>% 
    filter(TRB_count > 2) %>% 
    mutate(TRB_freq=TRB_count/TRA_top1_count) %>% slice_max(n=10, order_by=TRB_freq)

TRA[TRA$donor == "Gaydosik2022_MF17", ]

`summarise()` has grouped output by 'donor', 'TRA_top1', 'TRA_top1_count'. You can override using the `.groups` argument.


donor,TRA_top1,TRA_top1_count,TRB.1.cdr3,TRB_count,TRB_freq
<chr>,<fct>,<int>,<chr>,<int>,<dbl>
Gaydosik2022_MF17,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CAMSQAGTALIF,3185,NA,66,0.02072214


In [34]:
clonotypes_long <- full_join(TRB, TRA)
clonotypes_long$TRA_freq[is.na(clonotypes_long$TRA_freq)] <- 1
clonotypes_long$TRB_freq[is.na(clonotypes_long$TRB_freq)] <- 1
clonotypes_long$TRA_top1_count[is.na(clonotypes_long$TRA_top1_count)] <- 0
clonotypes_long$TRB_top1_count[is.na(clonotypes_long$TRB_top1_count)] <- 0

clonotypes_long[clonotypes_long$donor == "Gaydosik2022_MF17",]

Joining with `by = join_by(donor)`
Warning message in full_join(TRB, TRA):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


donor,TRB_top1,TRB_top1_count,TRA.1.cdr3,TRA_count,TRA_freq,TRA_top1,TRA_top1_count,TRB.1.cdr3,TRB_count,TRB_freq
<chr>,<fct>,<dbl>,<chr>,<int>,<dbl>,<fct>,<dbl>,<chr>,<int>,<dbl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,NA,66,0.02072214
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333,CAMSQAGTALIF,3185,NA,66,0.02072214


In [36]:
clonotypes_long %<>% 
    filter(TRA_freq > 0.02 &
           TRB_freq > 0.02 &
           (TRA_freq > 0.25 | TRB_freq > 0.25) & 
           TRA_freq+TRB_freq > 0.4 & 
           TRA_top1_count+TRB_top1_count > 25)

clonotypes_long[clonotypes_long$donor == "Gaydosik2022_MF17",]

donor,TRB_top1,TRB_top1_count,TRA.1.cdr3,TRA_count,TRA_freq,TRA_top1,TRA_top1_count,TRB.1.cdr3,TRB_count,TRB_freq
<chr>,<fct>,<dbl>,<chr>,<int>,<dbl>,<fct>,<dbl>,<chr>,<int>,<dbl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,NA,66,0.02072214
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006


In [37]:
subset <- which(clonotypes_long$TRA_freq > 0.95 & is.na(clonotypes_long$TRA.1.cdr3))
if(length(subset) > 0){
    clonotypes_long$TRA_top1[subset] <- NA
    clonotypes_long$TRA_top1_count[subset] <- 0
    clonotypes_long$TRB.1.cdr3[subset] <- NA
    clonotypes_long$TRB_count[subset] <- 0
    clonotypes_long$TRB_freq[subset] <- 0
}
clonotypes_long[clonotypes_long$donor == "Gaydosik2022_MF17",]

donor,TRB_top1,TRB_top1_count,TRA.1.cdr3,TRA_count,TRA_freq,TRA_top1,TRA_top1_count,TRB.1.cdr3,TRB_count,TRB_freq
<chr>,<fct>,<dbl>,<chr>,<int>,<dbl>,<fct>,<dbl>,<chr>,<dbl>,<dbl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,NA,66,0.02072214
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006


In [38]:
subset <- which(clonotypes_long$TRB_freq > 0.95 & is.na(clonotypes_long$TRB.1.cdr3))
if(length(subset) > 0){
    clonotypes_long$TRB_top1[subset] <- NA
    clonotypes_long$TRB_top1_count[subset] <- 0
    clonotypes_long$TRA.1.cdr3[subset] <- NA
    clonotypes_long$TRA_count[subset] <- 0
    clonotypes_long$TRA_freq[subset] <- 0
}

clonotypes_long[clonotypes_long$donor == "Gaydosik2022_MF17",]

donor,TRB_top1,TRB_top1_count,TRA.1.cdr3,TRA_count,TRA_freq,TRA_top1,TRA_top1_count,TRB.1.cdr3,TRB_count,TRB_freq
<chr>,<fct>,<dbl>,<chr>,<int>,<dbl>,<fct>,<dbl>,<chr>,<dbl>,<dbl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,CAMSQAGTALIF,3112,0.8644444,CAMSQAGTALIF,3185,NA,66,0.02072214
Gaydosik2022_MF17,CASSPTGQGAQETQYF,3600,NA,480,0.1333333,CAMSQAGTALIF,3185,CASSPTGQGAQETQYF,3112,0.97708006


In [39]:
subset <- clonotypes_long$TRB_top1_count/clonotypes_long$TRB_count > 10 & is.na(clonotypes_long$TRB.1.cdr3)
clonotypes_long$TRB.1.cdr3[subset] <- as.character(clonotypes_long$TRB_top1[subset])
subset <- clonotypes_long$TRA_top1_count/clonotypes_long$TRA_count > 10 & is.na(clonotypes_long$TRA.1.cdr3)
clonotypes_long$TRA.1.cdr3[subset] <- as.character(clonotypes_long$TRA_top1[subset])

In [40]:
clonotypes <- clonotypes_long %>% group_by(donor) %>% summarize(Malignant.TRA=paste(unique(na.omit(TRA.1.cdr3)), collapse=","), 
                                                                 Malignant.TRB=paste(unique(na.omit(TRB.1.cdr3)), collapse=","))

clonotypes %>% filter(grepl("Gaydosik", donor))

donor,Malignant.TRA,Malignant.TRB
<chr>,<chr>,<chr>
Gaydosik2022_MF17,CAMSQAGTALIF,CASSPTGQGAQETQYF
Gaydosik2022_MF18,CAMVPKWGGSYIPTF,CAISESDRDRVAFF
Gaydosik2022_MF19,"CALRKRGNTPLVF,CHGSSNTGKLIF",CASKLFMDREKGEAFF
Gaydosik2022_MF21,CVVTRTGGSYIPTF,CASSHRQGAISPLHF
Gaydosik2022_MF24,CAAPNSGGYQKVTF,CASSIYGYSYNSPLHF


In [41]:
clonotypes_TRB <- clonotypes %>% select(donor, Malignant.TRB) %>% 
    separate(Malignant.TRB, into=c("TRB.1", "TRB.2", "TRB.3"), sep=",") %>% 
    pivot_longer(c(TRB.1, TRB.2, TRB.3), values_to="TRB.1.cdr3") %>% 
    filter(!is.na(TRB.1.cdr3) & TRB.1.cdr3 != "") %>% select(donor, TRB.1.cdr3) %>% 
    mutate(Malignant.TRB=TRUE)

clonotypes_TRB %>% filter(grepl("Gaydosik", donor))

Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 17 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17].”


donor,TRB.1.cdr3,Malignant.TRB
<chr>,<chr>,<lgl>
Gaydosik2022_MF17,CASSPTGQGAQETQYF,TRUE
Gaydosik2022_MF18,CAISESDRDRVAFF,TRUE
Gaydosik2022_MF19,CASKLFMDREKGEAFF,TRUE
Gaydosik2022_MF21,CASSHRQGAISPLHF,TRUE
Gaydosik2022_MF24,CASSIYGYSYNSPLHF,TRUE


In [42]:
clonotypes_TRA <- clonotypes %>% select(donor, Malignant.TRA) %>% 
    separate(Malignant.TRA, into=c("TRA.1", "TRA.2", "TRA.3"), sep=",") %>% 
    pivot_longer(c(TRA.1, TRA.2, TRA.3), values_to="TRA.1.cdr3") %>% 
    filter(!is.na(TRA.1.cdr3) & TRA.1.cdr3 != "") %>% select(donor, TRA.1.cdr3) %>% 
    mutate(Malignant.TRA=TRUE)

clonotypes_TRA %>% filter(grepl("Gaydosik", donor))

Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 16 rows [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17].”


donor,TRA.1.cdr3,Malignant.TRA
<chr>,<chr>,<lgl>
Gaydosik2022_MF17,CAMSQAGTALIF,TRUE
Gaydosik2022_MF18,CAMVPKWGGSYIPTF,TRUE
Gaydosik2022_MF19,CALRKRGNTPLVF,TRUE
Gaydosik2022_MF19,CHGSSNTGKLIF,TRUE
Gaydosik2022_MF21,CVVTRTGGSYIPTF,TRUE
Gaydosik2022_MF24,CAAPNSGGYQKVTF,TRUE


In [43]:
coldata %<>% .[,grep("^Malignant.TR[AB]$", colnames(.), invert=TRUE)]

coldata %<>% rownames_to_column("barcode") 
coldata %<>% left_join(clonotypes_TRA)
coldata %<>% left_join(clonotypes_TRB)
coldata %<>% column_to_rownames("barcode") 

Joining with `by = join_by(donor, TRA.1.cdr3)`
Joining with `by = join_by(donor, TRB.1.cdr3)`


In [44]:
coldata$Malignant <- "Unknown"
coldata$Malignant[(coldata$Malignant.TRA == TRUE | coldata$Malignant.TRB == TRUE) & 
                  coldata$cell_type %in% c("CD4+ T-cells", "CD8+ T-cells", "NK cells", "NK_cell")] <- "Malignant"

coldata$Malignant[which(coldata$cell_type %in% c("CD4+ T-cells", "CD8+ T-cells") & 
                        coldata$Malignant != "Malignant" & 
                        (!is.na(coldata$TRA.1.cdr3) | !is.na(coldata$TRB.1.cdr3)))] <- "T cell"

In [45]:
table(coldata$Malignant)


Malignant    T cell   Unknown 
    33194     21998    204735 

# Prepare anndata object for scVI

In [26]:
# filter doublets from object
coldata_subset <- with(coldata, which(sample != "Doublet" & doublet != TRUE & scDblFinder.class != "doublet"))

In [27]:
library(reticulate)
library(anndata)

use_condaenv(condaenv='scvi', required=TRUE)
sc <- import('scanpy', convert = FALSE)
anndata <- import('anndata', convert = FALSE)
pd <- import('pandas', convert = FALSE)


Attaching package: ‘anndata’


The following object is masked from ‘package:readr’:

    read_csv




Add sample metadata to object

In [28]:
#metadata <- readxl::read_xlsx(here::here("data", "CTCL_MF_atlas_samples.xlsx"))
metadata <- read.table(here::here("data", "CTCL_MF_atlas_samples.tsv"), sep="\t", header=TRUE)
metadata %<>% group_by(run) %>% summarize(across(dplyr::everything(), function(x)x[1])) %>% column_to_rownames("run")

coldata %<>% cbind(metadata[as.character(.$run), c("chemistry", "lesion", "location", "CTCL_stage", "TNMB", "age", "sex")])

In [ ]:
adata <- AnnData(X=t(data_counts[, coldata_subset]), 
                     obs=coldata[coldata_subset, ] %>% .[,grep("^TR[ABGD]$", colnames(.), invert=TRUE)] %>% as.data.frame())
                     
write_h5ad(adata, here::here("data", "adata", paste0("all", ".h5ad")))

Make SingleCellObject as well and do batch normalization

In [ ]:
sce_list <- imap(counts_list, function(counts, run){
    sce <- SingleCellExperiment(list(counts=scutility::subset_matrix(counts, features=rownames_include)))
    colData(sce) <- DataFrame(coldata[colnames(counts), ])
    
    return(sce)
})

In [ ]:
library(BiocParallel)
sce_merged <- batchelor::multiBatchNorm(sce_list, preserve.single=TRUE, BPPARAM=MulticoreParam(workers=32)) %>% 
            Reduce(cbind, .)

In [ ]:
sce_merged

In [ ]:
mainExpName(sce_merged) <- "RNA"
multisce_save(sce_merged, path=here::here("data", "multisce", "all"), main_name="RNA")